In [1]:
#-------------------------------------------------------------------------------
# Define all required libraries, routines, and modules
#-------------------------------------------------------------------------------
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import sys
import mpl_toolkits.basemap
import os
#import pyroms
sys.path.append('/Users/actodd/MYPYTHON/seawater/')
import seawater as sw

#-------------------------------------------------------------------------------
# Define all required sub-libraries, sub-routines, and sub-modules
#-------------------------------------------------------------------------------
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from matplotlib import cm
#from pyroms.depths import nc_depths


In [4]:
#--------------------------------------------------------------------------
# Routine for calculating depths at rho points
#--------------------------------------------------------------------------
def depths_rho(h,zeta,s_r,Cs_r,hc):
  for k in range(0,len(s_r)):
    z0=(hc*s_r[k]+Cs_r[k]*h)/(hc+h);
    z[k,:,:]=zeta+(zeta+h)*z0;
  return z

In [3]:
#--------------------------------------------------------------------------
# Routine for calculating depths at u points
#--------------------------------------------------------------------------
def depths_u(h,zeta,s_r,Cs_r,hc):
  M=len(h)
  L=len(h[0])
  hu=   0.5*(   h[0:M-1,1:L-1]+   h[0:M-1,0:L-2]);
  zetau=0.5*(zeta[0:M-1,1:L-1]+zeta[0:M-1,0:L-2])
  
  # Loop through each depth layer
  for k in range(0,len(s_r)):
    z0=(hc*s_r[k]+Cs_r[k]*hu)/(hc+hu);
    z[k,:,:]=zetau+(zetau+hu)*z0;
  #  endfor
  
  del hu
  del zetau
  return z

In [5]:
#--------------------------------------------------------------------------
# Routine for calculating depths at v points
#--------------------------------------------------------------------------
def depths_v(h,zeta,s_r,Cs_r,hc):
  M=len(h)
  L=len(h[0])
  hv=   0.5*(   h[1:M-1,0:L-1]+   h[0:M-2,0:L-1]);
  zetav=0.5*(zeta[1:M-1,0:L-1]+zeta[0:M-2,0:L-1])
  
  # Loop through all layers
  for k in range(0,len(s_r)):
    z0=(hc*s_r[k]+Cs_r[k]*hv)/(hc+hv);
    z[k,:,:]=zetav+(zetav+hv)*z0;
  #  endfor
  del hv
  del zetav
  return z

In [6]:
#--------------------------------------------------------------------------
# Routine for calculating depths at w points
#--------------------------------------------------------------------------
def depths_w(h,zeta,s_w,Cs_w,hc):
  for k in range(0,len(s_w)):
    z0=(hc*s_w[k]+Cs_w[k]*h)/(hc+h);
    zw[k,:,:]=zeta+(zeta+h)*z0;
  return zw

In [23]:
################################################################################
#                                                                              #
#                            User Defined Variables                            #
#                                                                              #
input_file  = '/Volumes/Black_box/Data/USeast-age/output/clim/averages/'+\
              'avg_3hrly.nc'
MLD_file    = '/Volumes/Black_box/Data/USeast-age/output/clim/averages/'+\
              'age_pycnocline_avg.nc'
Grtime_file = '/Volumes/Black_box/Data/USeast-rtime/output/GOM_shelf/'+\
              'rtime_GOM_shelf.nc'
Artime_file = '/Volumes/Black_box/Data/USeast-rtime/output/ATL/'+\
              'rtime_ATL.nc'
ATL_file    = '/Volumes/Black_box/Data/USeast/Data/grd/grid_ATLscope.nc'
GOM_file    = '/Volumes/Black_box/Data/USeast/Data/grd/grid_GOM_shelf_scope.nc'
grid_file   = '/Volumes/Black_box/Data/USeast/Data/grd/USeast-grid.nc'
output_file = '/Volumes/Black_box/Data/USeast-age/output/clim/averages/'+\
              'transit_pycnocline.nc'
#                                                                              #
################################################################################

In [15]:
#-------------------------------------------------------------------------------
# Open grid file and read in variables
#-------------------------------------------------------------------------------
grid_data=Dataset(grid_file,mode='r')
mask=grid_data.variables['mask_rho' ][:,:]
lon =grid_data.variables['lon_rho'  ][:,:]
lat =grid_data.variables['lat_rho'  ][:,:]
h   =grid_data.variables['h'        ][:,:]
grid_data.close()

hc=200.

grid_data=Dataset(ATL_file,mode='r')
hmask  = grid_data.variables['mask_rho' ][:]
Ascope = grid_data.variables['scope_rho'][:]
grid_data.close()

# Get GOM Shelf gird
grid_data=Dataset(GOM_file,mode='r')
Gscope = grid_data.variables['scope_rho'][:]
grid_data.close()

scope=Ascope+Gscope


In [9]:
#-------------------------------------------------------------------------------
# Read in additional grid variables from forward file
#-------------------------------------------------------------------------------
input_data=Dataset(input_file,mode='r')
s_r =input_data.variables['s_rho'][:]
s_w =input_data.variables['s_w'  ][:]
Cs_r=input_data.variables['Cs_r' ][:]
Cs_w=input_data.variables['Cs_w' ][:]

L=len(input_data.dimensions['xi_rho' ])
M=len(input_data.dimensions['eta_rho'])
N=len(input_data.dimensions['s_rho'  ])

In [14]:
time_out = input_data.variables['ocean_time'][:]

In [21]:
scope[440,380]

1.0

In [22]:
#-------------------------------------------------------------------------------
# Open residence time files
#-------------------------------------------------------------------------------
Atime_data =Dataset(Artime_file,mode='r')
Gtime_data =Dataset(Grtime_file,mode='r')


In [24]:
#-------------------------------------------------------------------------------
# Open residence time files
#-------------------------------------------------------------------------------
MLD_data =Dataset(MLD_file,mode='r')


In [29]:
#------------------------------------------------------------------------
# Load in the SSH, Temp, and Salt to calculate layer & thermocline depths
#------------------------------------------------------------------------
zeta=input_data.variables['zeta'][0,  :,:]
temp=input_data.variables['temp'][0,:,:,:]
salt=input_data.variables['salt'][0,:,:,:]
MLD =  MLD_data.variables['MLD' ][0,  :,:]

#------------------------------------------------------------------------
# Calculate depths of each rho layer
#------------------------------------------------------------------------
z  = np.zeros(shape=(N  ,M,L))
zw = np.zeros(shape=(N+1,M,L))
z  = depths_rho(h,zeta,s_r,Cs_r,hc)
zw = depths_w(  h,zeta,s_w,Cs_w,hc)
Hz = abs(zw[1:N+1,:,:]-zw[0:N,:,:])


In [30]:
#------------------------------------------------------------------------
# Calculate various seawater properties
#------------------------------------------------------------------------
pres=sw.pres(z*-1,lat)
dens=sw.dens(salt,temp,pres)-1000.


In [31]:
drdZ = abs(dens[1:N-1,440,380]-dens[0:N-2,440,380])/abs(z[1:N-1,440,380]-z[0:N-2,440,380])
drdZ_ind=np.argmax(drdZ)


In [32]:
MLD[440,380]

70.93837

In [33]:
abs(   z[drdZ_ind,   440,380])

70.938367959104426

In [34]:
z[:,440,380]

array([-144.35308881, -139.36846471, -133.76543513, -127.79235994,
       -121.63304831, -115.42189366, -109.25561807, -103.20236199,
        -97.30869907,  -91.60503234,  -86.10973059,  -80.83228604,
        -75.77571419,  -70.93836796,  -66.31530092,  -61.89928447,
        -57.68156043,  -53.65239226,  -49.80146386,  -46.11816351,
        -42.59178208,  -39.2116475 ,  -35.96721263,  -32.84810912,
        -29.84417714,  -26.94547823,  -24.14229661,  -21.42513309,
        -18.78469441,  -16.21188024,  -13.69776935,  -11.23360599,
         -8.81078731,   -6.42085232,   -4.05547265,   -1.70644545])

In [38]:
min(abs(abs(z[:,440,380])-MLD[440,380]))

1.7918721368914703e-06

In [42]:
np.argmin(abs(abs(z[:,440,380])-MLD[440,380]))

13

In [40]:
zz[13]

1.7918721368914703e-06

In [41]:
z[13,440,380]

-70.938367959104426